In [1]:
# import os
# os.environ["R_HOME"] = "/home/ld/anaconda3/envs/r/lib/R"
import sys
sys.path.append('../')
import scDesign3

The R project used is located at D:\R-4.2.3


In [2]:
import anndata as ad
example_sce = ad.read_h5ad('test_data/VISIUM.h5ad')
example_sce.obs['test'] = ['test' for i in range(example_sce.n_obs)]
example_sce = example_sce[0:10,0:10]

In [3]:
example_sce

View of AnnData object with n_obs × n_vars = 10 × 10
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'spatial1', 'spatial2', 'cell_type', 'test'
    var: 'name'

In [4]:
test = scDesign3.scDesign3()

construct_data interface

In [14]:
## windows
bpparam = test.get_bpparam("SnowParam",stop_on_error = False)

## linux/mac
# bpparam = test.get_bpparam("MulticoreParam",stop_on_error = False)

class: SnowParam
  bpisup: FALSE; bpnworkers: 2; bptasks: 0; bpjobname: BPJOB
  bplog: FALSE; bpthreshold: INFO; bpstopOnError: FALSE
  bpRNGseed: ; bptimeout: NA; bpprogressbar: FALSE
  bpexportglobals: TRUE; bpexportvariables: TRUE; bpforceGC: FALSE; bpfallback: TRUE
  bplogdir: NA
  bpresultdir: NA
  cluster type: SOCK



In [6]:
const_data = test.construct_data(
    anndata=example_sce,
    corr_by="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
    parallelization='bpmapply',
    bpparam=bpparam
)

In [7]:
print(const_data)

$count_mat
                   Calb2 Gng4 Ttr S100a5 Nrgn Doc2g Ppp1r1b Pcbp3 Fabp7 Gpsm1
AAACAAGTATCTCCCA-1     0    0   2      0    2     1       2     1     0     0
AAACACCAATAACTGC-1    22    7   3     27    4    25       0     9    24     5
AAACAGAGCGACTCCT-1     0    1   1      0  156     0       2     4     0     0
AAACAGCTTTCAGAAG-1    16   43   2      0   30     3       0    24    17    24
AAACAGGGTCTATATT-1    19   27   2      0   15     2       1    16     7    27
AAACATGGTGAGAGGA-1    34    5   4     53    1     6       0     7    71    10
AAACATTTCCCGGATT-1     7    2   1      0   34     0       8     3     1     4
AAACCGGGTAGGTACC-1     3    2   3      0  213     4       7     4     4     1
AAACCGTTCGTCCAGG-1     2    0   4      3   16     1       5     2     2     3
AAACCTAAGCAGCCGG-1     4    0   2      0   99     1      88     4     4     8

$dat
                   cell_type spatial1 spatial2 corr_group
AAACAAGTATCTCCCA-1         2     7475     8501          1
AAACACCAA

In [8]:
# SummarizedExperiment = packages.importr('SummarizedExperiment')
# print(SummarizedExperiment.assay(test._sce,test._assay_use))
# print(SummarizedExperiment.colData(test._sce))
# print(SummarizedExperiment.rowData(test._sce))

fit_marginal interface

In [9]:
marginal = test.fit_marginal(
    data=const_data,
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    n_cores=3,
    usebam=False,
    parallelization='bpmapply',
    bpparam=bpparam,
    trace=True,
)

In [10]:
print(marginal)

$Calb2
$Calb2$fit

Family: Negative Binomial(2.63) 
Link function: log 

Formula:
gene ~ s(spatial1, spatial2, bs = "gp", k = 2)

Estimated degrees of freedom:
2  total = 3 

REML score: 42.72744     

$Calb2$warning
$Calb2$warning[[1]]
$Calb2$warning[[1]]$function_name
[1] "gam"

$Calb2$warning[[1]]$type
[1] "warning"

$Calb2$warning[[1]]$message
[1] "simpleWarning in smooth.construct.gp.smooth.spec(object, dk$data, dk$knots): basis dimension reset to minimum possible\n"



$Calb2$time
[1] 0.112381       NA

$Calb2$removed_cell
[1] NA


$Gng4
$Gng4$fit

Family: Negative Binomial(0.723) 
Link function: log 

Formula:
gene ~ s(spatial1, spatial2, bs = "gp", k = 2)

Estimated degrees of freedom:
2  total = 3 

REML score: 40.26233     

$Gng4$warning
$Gng4$warning[[1]]
$Gng4$warning[[1]]$function_name
[1] "gam"

$Gng4$warning[[1]]$type
[1] "warning"

$Gng4$warning[[1]]$message
[1] "simpleWarning in smooth.construct.gp.smooth.spec(object, dk$data, dk$knots): basis dimension reset to minim

fit_copula interface

In [19]:
copula = test.fit_copula(
    family_use="nb",
    n_cores=1,
    copula = "vine",
    # parallelization="bpmapply",
    # bpparam=bpparam,
    important_feature="all",
)

R[write to console]: Convert Residuals to Uniform

R[write to console]: Converting End

R[write to console]: Copula group 1 starts

R[write to console]: Vine Copula Estimation Starts



Time difference of 0.2581239 secs


R[write to console]: Vine Copula Estimation Ends



In [20]:
print(copula)

$model_aic
aic.marginal   aic.copula    aic.total 
    578.7325     -42.4411     536.2914 

$model_bic
bic.marginal   bic.copula    bic.total 
   591.68515    -39.11266    552.57248 

$copula_list
$copula_list$`1`
10-dimensional vine copula fit ('vinecop')
nobs = 10   logLik = 32.22   npars = 11   AIC = -42.44   BIC = -39.11   


$important_feature
 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE


